In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
!gc-monitor

+---------------+--------------------------------------------------------------+
|  gc-monitor   | Partition: 'mkbahk-pt-4ipus_ipuof' has 4 reconfigurable IPUs |
+-------------+--------------------+----------+------+--------------+----+---------+-------+-------+
|    IPU-M    |       Serial       |  ICU FW  | Type |Server version| ID | PCIe ID |Routing| GWSW  |
+-------------+--------------------+----------+------+--------------+----+---------+-------+-------+
|  10.1.5.10  | 0115.0002.8211021  |  2.3.2   |M2000 |    1.7.1     | 0  |    3    |  DNC  |       |
|  10.1.5.10  | 0115.0002.8211021  |  2.3.2   |M2000 |    1.7.1     | 1  |    2    |  DNC  |       |
+-------------+--------------------+----------+------+--------------+----+---------+-------+-------+
|  10.1.5.10  | 0115.0001.8211021  |  2.3.2   |M2000 |    1.7.1     | 2  |    1    |  DNC  |       |
|  10.1.5.10  | 0115.0001.8211021  |  2.3.2   |M2000 |    1.7.1     | 3  |    0    |  DNC  |       |
+-------------+---------------

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python import ipu

In [4]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

In [5]:
@tf.function(experimental_compile=True)
def matmul_fn(x, y):
    z = tf.matmul(x, y)
    return z
###end of def:

In [6]:
!gc-info -l

Graphcore device listing:

-+- Id: [0], target:    [Fabric], PCI Domain: [3]
-+- Id: [1], target:    [Fabric], PCI Domain: [2]
-+- Id: [2], target:    [Fabric], PCI Domain: [1]
-+- Id: [3], target:    [Fabric], PCI Domain: [0]
-+- Id: [4], target: [Multi IPU]
 |--- Id: [0], DNC Id: [0], PCI Domain: [3]
 |--- Id: [1], DNC Id: [1], PCI Domain: [2]
-+- Id: [5], target: [Multi IPU]
 |--- Id: [2], DNC Id: [0], PCI Domain: [1]
 |--- Id: [3], DNC Id: [1], PCI Domain: [0]
-+- Id: [6], target: [Multi IPU]
 |--- Id: [0], DNC Id: [0], PCI Domain: [3]
 |--- Id: [1], DNC Id: [1], PCI Domain: [2]
 |--- Id: [2], DNC Id: [2], PCI Domain: [1]
 |--- Id: [3], DNC Id: [3], PCI Domain: [0]



In [9]:
cfg = ipu.config.IPUConfig()
# Access by full name from the base config:
metadata = cfg.get_attribute_metadata("ipu_model.version")
# Access by name relative to the "ipu_model" sub-category:
metadata = cfg.ipu_model.get_attribute_metadata("version")

# Use the metadata
#print(metadata.types)  # [str]
print(metadata.default)  # "ipu2"
print(metadata.deprecated)  # False indicates it is not deprecated
print(metadata.__doc__)  # "Specify the ipu version to be used by the..."

# Check a value against the option's type
#metadata.check_type(5)  # Will fail, since this option needs a string.

# Print a deprecation message if the option is deprecated
metadata.warn_if_deprecated()
#cfg.auto_select_ipus = 4
#cfg.select_ipus = 6
cfg.select_ipus = [0, 1]
cfg.configure_ipu_system()

ipu2
False
Specify the IPU version to be used by the IPU Model. Options are "ipu1" or
"ipu2" (default).


In [10]:
strategy = ipu.ipu_strategy.IPUStrategy()
with strategy.scope():
    c = strategy.run(matmul_fn, args=(a, b))
###end of with:

In [11]:
print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
